In [26]:
import numpy as np
import random

# Đọc dữ liệu từ file
def read_flowshop_data(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        num_jobs, num_machines = map(int, lines[0].split())
        processing_times = np.zeros((num_jobs, num_machines), dtype=int)

        for i, line in enumerate(lines[1:]):
            data = list(map(int, line.split()))
            processing_times[i] = [data[j + 1] for j in range(0, len(data), 2)]

        return num_jobs, num_machines, processing_times


# Tính thời gian hoàn thành (makespan)
def calculate_makespan(sequence, processing_times):
    num_jobs = len(sequence)
    num_machines = processing_times.shape[1]
    completion_time = np.zeros((num_jobs, num_machines), dtype=int)

    # Tính thời gian hoàn thành cho từng công việc
    for i, job in enumerate(sequence):
        for machine in range(num_machines):
            if i == 0 and machine == 0:
                completion_time[i][machine] = processing_times[job][machine]
            elif i == 0:
                completion_time[i][machine] = completion_time[i][machine - 1] + processing_times[job][machine]
            elif machine == 0:
                completion_time[i][machine] = completion_time[i - 1][machine] + processing_times[job][machine]
            else:
                completion_time[i][machine] = max(completion_time[i - 1][machine], completion_time[i][machine - 1]) + processing_times[job][machine]

    return completion_time[-1][-1]


# Thuật toán ACO
def aco_flowshop(num_jobs, processing_times, num_ants=10, alpha=1.0, beta=2.0, evaporation=0.5, num_iterations=100):
    pheromone = np.ones((num_jobs, num_jobs))
    best_sequence = None
    best_makespan = float('inf')

    for iteration in range(num_iterations):
        all_sequences = []
        all_makespans = []

        # Các con kiến tạo ra các giải pháp
        for ant in range(num_ants):
            sequence = []
            available_jobs = list(range(num_jobs))

            while available_jobs:
                if len(sequence) == 0:
                    # Chọn công việc đầu tiên ngẫu nhiên
                    next_job = random.choice(available_jobs)
                else:
                    # Tính xác suất chọn các công việc tiếp theo
                    last_job = sequence[-1]
                    probabilities = [
                        (job, (pheromone[last_job][job] ** alpha) * ((1.0 / processing_times[job].sum()) ** beta))
                        for job in available_jobs
                    ]
                    total_prob = sum(prob for _, prob in probabilities)
                    probabilities = [(job, prob / total_prob) for job, prob in probabilities]
                    next_job = random.choices(
                        [job for job, _ in probabilities],
                        weights=[prob for _, prob in probabilities]
                    )[0]

                sequence.append(next_job)
                available_jobs.remove(next_job)

            # Tính makespan cho giải pháp này
            makespan = calculate_makespan(sequence, processing_times)
            all_sequences.append(sequence)
            all_makespans.append(makespan)

            # Cập nhật giải pháp tốt nhất
            if makespan < best_makespan:
                best_sequence, best_makespan = sequence, makespan

        # Cập nhật pheromone
        pheromone *= (1 - evaporation)
        for sequence, makespan in zip(all_sequences, all_makespans):
            for i in range(len(sequence) - 1):
                pheromone[sequence[i]][sequence[i + 1]] += 1.0 / makespan

    return best_sequence, best_makespan


# Hàm chính
if __name__ == "__main__":
    filename = "data3.txt"  # Đổi tên file nếu cần
    num_jobs, num_machines, processing_times = read_flowshop_data(filename)

    best_sequence, best_makespan = aco_flowshop(num_jobs, processing_times)
    print("Best sequence:", best_sequence)
    print("Best makespan:", best_makespan)


Best sequence: [18, 17, 3, 19, 15, 14, 1, 4, 2, 13, 16, 0, 9, 7, 11, 10, 6, 8, 12, 5]
Best makespan: 1348


In [38]:
import numpy as np
import random


def read_flowshop_data(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        num_jobs, num_machines = map(int, lines[0].split())
        processing_times = np.zeros((num_jobs, num_machines), dtype=int)

        for i, line in enumerate(lines[1:]):
            data = list(map(int, line.split()))
            processing_times[i] = [data[j + 1] for j in range(0, len(data), 2)]

        return num_jobs, num_machines, processing_times


def calculate_makespan(sequence, processing_times):
    num_jobs = len(sequence)
    num_machines = processing_times.shape[1]
    completion_time = np.zeros((num_jobs, num_machines), dtype=int)

    for i, job in enumerate(sequence):
        for machine in range(num_machines):
            if i == 0 and machine == 0:
                completion_time[i][machine] = processing_times[job][machine]
            elif i == 0:
                completion_time[i][machine] = completion_time[i][machine - 1] + processing_times[job][machine]
            elif machine == 0:
                completion_time[i][machine] = completion_time[i - 1][machine] + processing_times[job][machine]
            else:
                completion_time[i][machine] = max(completion_time[i - 1][machine], completion_time[i][machine - 1]) + processing_times[job][machine]

    return completion_time[-1][-1]


def two_opt_swap(sequence, processing_times):
    best_sequence = sequence[:]
    best_makespan = calculate_makespan(best_sequence, processing_times)

    for i in range(len(sequence) - 1):
        for j in range(i + 1, len(sequence)):
            new_sequence = sequence[:]
            new_sequence[i], new_sequence[j] = new_sequence[j], new_sequence[i]
            new_makespan = calculate_makespan(new_sequence, processing_times)

            if new_makespan < best_makespan:
                best_sequence, best_makespan = new_sequence, new_makespan

    return best_sequence, best_makespan


def aco_flowshop(num_jobs, processing_times, num_ants=10, alpha=1.0, beta=2.0, evaporation=0.3, num_iterations=100):
    pheromone = np.ones((num_jobs, num_jobs))
    best_sequence = None
    best_makespan = float('inf')

    for iteration in range(num_iterations):
        all_sequences = []
        all_makespans = []

        for ant in range(num_ants):
            sequence = []
            available_jobs = list(range(num_jobs))

            while available_jobs:
                if len(sequence) == 0:
                    next_job = random.choice(available_jobs)
                else:
                    last_job = sequence[-1]
                    probabilities = [
                        (job, (pheromone[last_job][job] ** alpha) * ((1.0 / calculate_makespan(sequence + [job], processing_times)) ** beta))
                        for job in available_jobs
                    ]
                    total_prob = sum(prob for _, prob in probabilities)
                    probabilities = [(job, prob / total_prob) for job, prob in probabilities]
                    next_job = random.choices(
                        [job for job, _ in probabilities],
                        weights=[prob for _, prob in probabilities]
                    )[0]

                sequence.append(next_job)
                available_jobs.remove(next_job)

            # Local search
            sequence, makespan = two_opt_swap(sequence, processing_times)
            all_sequences.append(sequence)
            all_makespans.append(makespan)

            if makespan < best_makespan:
                best_sequence, best_makespan = sequence, makespan

        # Update pheromone using the best solution
        pheromone *= (1 - evaporation)
        for i in range(len(best_sequence) - 1):
            pheromone[best_sequence[i]][best_sequence[i + 1]] += 1.0 / best_makespan

    return best_sequence, best_makespan


if __name__ == "__main__":
    filename = "data3.txt"
    num_jobs, num_machines, processing_times = read_flowshop_data(filename)

    best_sequence, best_makespan = aco_flowshop(num_jobs, processing_times)
    print("Best sequence:", best_sequence)
    print("Best makespan:", best_makespan)


Best sequence: [3, 15, 13, 12, 7, 10, 9, 14, 4, 6, 16, 19, 1, 18, 8, 17, 11, 0, 2, 5]
Best makespan: 1323


Best sequence: [3, 15, 8, 5, 7, 18, 16, 11, 14, 19, 0, 17, 4, 10, 9, 13, 6, 12, 2, 1]
Best makespan: 1302

In [47]:
# import numpy as np
# import random

# # Đọc dữ liệu từ file
# def read_input(file_path):
#     with open(file_path, 'r') as f:
#         # Đọc số lượng công việc và máy từ dòng đầu tiên
#         num_jobs, num_machines = map(int, f.readline().split())
        
#         # Khởi tạo ma trận để lưu thời gian xử lý của các công việc trên các máy
#         processing_times = np.zeros((num_jobs, num_machines), dtype=int)
        
#         # Đọc từng dòng tiếp theo và điền vào ma trận
#         for i in range(num_jobs):
#             data = list(map(int, f.readline().split()))
#             for j in range(num_machines):
#                 processing_times[i, j] = data[2 * j + 1]  # Lấy các thời gian ở các vị trí 1, 3, 5, ...
    
#     return processing_times


# # Hàm tính makespan từ một thứ tự công việc
# def calculate_makespan(order, processing_times):
#     num_jobs, num_machines = processing_times.shape
#     completion_times = np.zeros((num_jobs, num_machines), dtype=int)

#     for i, job in enumerate(order):
#         for j in range(num_machines):
#             if i == 0 and j == 0:
#                 completion_times[i, j] = processing_times[job, j]
#             elif i == 0:
#                 completion_times[i, j] = completion_times[i, j-1] + processing_times[job, j]
#             elif j == 0:
#                 completion_times[i, j] = completion_times[i-1, j] + processing_times[job, j]
#             else:
#                 completion_times[i, j] = max(completion_times[i-1, j], completion_times[i, j-1]) + processing_times[job, j]
    
#     return completion_times[-1, -1]  # Returns the makespan


# # Hàm NEH để sắp xếp các công việc
# def neh(processing_times):
#     num_jobs, num_machines = processing_times.shape
#     jobs = list(range(num_jobs))
    
#     # Sắp xếp các công việc dựa trên tổng thời gian xử lý
#     sorted_jobs = sorted(jobs, key=lambda x: sum(processing_times[x, :]), reverse=True)
    
#     # NEH: Xây dựng thứ tự công việc từ đầu đến cuối
#     current_order = [sorted_jobs[0]]
#     for i in range(1, num_jobs):
#         best_makespan = float('inf')
#         best_order = None
#         for j in range(i + 1):
#             new_order = current_order[:j] + [sorted_jobs[i]] + current_order[j:]
#             makespan = calculate_makespan(new_order, processing_times)
#             if makespan < best_makespan:
#                 best_makespan = makespan
#                 best_order = new_order
#         current_order = best_order

#     return current_order


# # Hàm tính mật độ pheromone và heuristic
# def initialize_pheromone_and_heuristic(num_jobs):
#     pheromone = np.ones((num_jobs, num_jobs))
#     heuristic = np.ones((num_jobs, num_jobs))  # Sử dụng giá trị heuristic đơn giản là 1 cho tất cả các cặp công việc
#     return pheromone, heuristic


# # Thuật toán ACO kết hợp với NEH
# # Thuật toán ACO kết hợp với NEH
# def aco_with_neh(processing_times, num_ants=10, alpha=1.0, beta=2.0, evaporation=0.3, iterations=100):
#     num_jobs = len(processing_times)
#     pheromone, heuristic = initialize_pheromone_and_heuristic(num_jobs)
    
#     best_solution = None
#     best_makespan = float('inf')
    
#     for _ in range(iterations):
#         solutions = []
#         makespans = []

#         # Mỗi con kiến tìm kiếm một giải pháp
#         for _ in range(num_ants):
#             # Sử dụng NEH để tạo một giải pháp ban đầu
#             initial_order = neh(processing_times)

#             # Áp dụng thuật toán ACO để cải thiện giải pháp
#             current_solution = initial_order[:]
#             current_makespan = calculate_makespan(current_solution, processing_times)
#             solutions.append(current_solution)
#             makespans.append(current_makespan)

#             # Cập nhật pheromone dựa trên makespan
#             pheromone_delta = np.zeros_like(pheromone)
#             for i in range(num_jobs - 1):
#                 pheromone_delta[current_solution[i], current_solution[i + 1]] += 1 / current_makespan
            
#             pheromone += pheromone_delta * evaporation  # Cập nhật pheromone

#         # Tìm giải pháp tốt nhất trong vòng lặp này
#         best_ant_index = np.argmin(makespans)
#         if makespans[best_ant_index] < best_makespan:
#             best_makespan = makespans[best_ant_index]
#             best_solution = solutions[best_ant_index]

#         # Cập nhật pheromone với các giá trị tốt nhất
#         pheromone = (1 - evaporation) * pheromone
    
#     return best_solution, best_makespan


# # Kiểm tra kết quả
# if __name__ == "__main__":
#     file_path = "data1.txt"  # Thay bằng đường dẫn file của bạn
#     processing_times = read_input(file_path)
#     best_solution, best_makespan = aco_with_neh(processing_times)
    
#     print("Best solution:", best_solution)
#     print("Best makespan:", best_makespan)


Best solution: [7, 4, 2, 0, 3, 5, 1, 6]
Best makespan: 5466
